In [22]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE
import copy
from anytree import Node, RenderTree, LevelOrderGroupIter, PreOrderIter

# DSL.pptx DSLAndXtext.pptx ModelManagementXtend1.pptx ModelManagementXtend2.pptx SPLVariability-light1718.pptx
prs = Presentation("/Users/macher1/Google Drive/TEACHING/MDE1819/teaching-MDE-IL1819/slides-pptx/DSL.pptx")

# text_runs will be populated with a list of strings,
# one for each text run in presentation
text_runs = []

strOutput = ""


def gen_output(strO, newline=True):
    #print(strO)
    global strOutput 
    strOutput = strOutput + strO 
    if (newline):
        strOutput =  strOutput + "\n"

def findLastParent(tr, level):
    node_levels = [[node for node in children] for children in LevelOrderGroupIter(tr)][level]
    return node_levels[len(node_levels) - 1]


def renderNode(n):
    if (n.font is None):
        return el.name
    si = n.font.size
    if (si is not None):
        if (si.pt >= 48):
            return "# " + el.name
        elif (si.pt >= 24 and si.pt < 48):
            return "## " + el.name
        elif (si.pt >= 18):
            return "### " + el.name
        else:
            return el.name
    else:
        return el.name

def is_float(value):
  try:
    float(value)
    return True
  except:
    return False

# debugging
def mkFont(ft): 
    if (ft.size):
        return str(ft.size.pt) + str(ft.name)
    return str(ft.size) + str(ft.name)

# deprecated
def node2listHTML(n):    
    gen_output("<li>")
    gen_output(n.name)
    gen_output(mkFont(n.font))
    gen_output("</li>")
    if n.is_leaf:
        return
    if (len(n.children) >= 1):
            gen_output("<ul>")
            for ch in n.children:
                node2list(ch)
            gen_output("</ul>")

# TODO Scala repeat space in Python?
def node2list(n, level=0): 
    if (level == 0):
        gen_output(" * " + n.name)
    elif (level == 1):
        gen_output("   * " + n.name)
    elif (level == 2):
        gen_output("     * " + n.name)
    elif (level == 3):
        gen_output("       * " + n.name)
    if n.is_leaf:
        return
    if (len(n.children) >= 1):
            for ch in n.children:
                node2list(ch, level + 1)

npage = 1
nimage = 0
for slide in prs.slides:
    #gen_output("class: middle, center, title-slide")
    npage = npage + 1
    
    for shape in slide.shapes:

        if shape.shape_type == MSO_SHAPE_TYPE.PICTURE: 
            image = shape.image
            # ---get image "file" contents---
            image_bytes = image.blob
            # ---make up a name for the file, e.g. 'image.jpg'---
            nimage = nimage + 1
            image_filename = 'imgs/image%d.%s' % (nimage, image.ext)
            with open(image_filename, 'wb') as f:
                f.write(image_bytes)
            #size = image.size
            width = str(shape.width.pt) #str(size[0]) # str(size[0])
            height = str(shape.height.pt) # str(size[1])
            gen_output('<img src="' + image_filename + '"' + ' width=' + width + ' height=' + height + ' />')
            #TODO: width/height
            #gen_output('.width-10[![](' + image_filename + ')]')
        
        if not shape.has_text_frame:
            continue
        
        old_level = 0
        myList = Node("top")
        for paragraph in shape.text_frame.paragraphs: 
            
            # only for gathering the font!
            for run in paragraph.runs:
                if (not run.text == ''):                    
                    text_runs.append(run.text)
                    #if (run.text == "IL1819"):
                        # print (run.text)
                    #    run.text = "IL1920"
                    cfont = run.font
            if not paragraph.text == '':
                
                if paragraph.level == 0:                    
                    Node (paragraph.text, parent=myList, font=cfont)
                else: 
                    if paragraph.level > old_level:
                        p = findLastParent(myList, paragraph.level)
                        n = Node (paragraph.text, parent=p, font=cfont)                         
                    elif old_level == paragraph.level:
                        p = findLastParent(myList, old_level)
                        Node (paragraph.text, parent=p, font=cfont)                        
                old_level = paragraph.level
               
        #for pre, fill, node in RenderTree(myList):
        #    print("%s%s" % (pre, node.name))

        #print("\n")
    
        r = myList.root  
        for el in r.children:            
            #print(el)
            if (len(el.siblings) == 0):
                if (is_float(el.name)): # page number 
                    continue
                else: 
                    gen_output(renderNode(el))                
            else:    
                node2list(el, 0)
    
    gen_output("---") # end of page/slide
            

#print(strOutput)
with open("result.md", 'w') as outf:
    outf.write(strOutput)
    outf.flush()
#prs.save('test.pptx')


In [ ]:
#text_runs

In [ ]:


def iter_picture_shapes(prs):
    for slide in prs.slides:
        for shape in slide.shapes:
            if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                yield shape

i = 0
for picture in iter_picture_shapes(prs):
    image = picture.image
    # ---get image "file" contents---
    image_bytes = image.blob
    # ---make up a name for the file, e.g. 'image.jpg'---
    i = i + 1
    image_filename = 'image%d.%s' % (i, image.ext)
    with open(image_filename, 'wb') as f:
        f.write(image_bytes)

In [ ]:
from anytree import Node, RenderTree, LevelOrderGroupIter, PreOrderIter

udo = Node("Udo")
marc = Node("Marc", parent=udo)
lian = Node("Lian", parent=marc)
dan = Node("Dan", parent=udo)
jet = Node("Jet", parent=dan)
jan = Node("Jan", parent=dan)
joe = Node("Joe", parent=dan)

print(udo)
Node('/Udo')
print(joe)
Node('/Udo/Dan/Joe')

for pre, fill, node in RenderTree(udo):
    print("%s%s" % (pre, node.name))



#findLastParent(udo, 1)
    
    
    

    
